In [17]:
pip install numpy==1.26.4 --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 60.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [1]:
# Install required packages
!pip install flaml pandas scikit-learn joblib

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
from flaml import AutoML

# -------------------------------
# Load dataset
# -------------------------------
data_path = "/content/diabetes.csv"
df = pd.read_csv(data_path)

# -------------------------------
# Inspect data
# -------------------------------
print("Sample records:")
print(df.head())

# -------------------------------
# Clean Data
# -------------------------------
# Replace 0 in columns where 0 is invalid with NaN
cols_with_zero_invalid = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_zero_invalid] = df[cols_with_zero_invalid].replace(0, np.nan)

# Fill missing values with median
df.fillna(df.median(), inplace=True)

# -------------------------------
# Separate features and target
# -------------------------------
X = df.drop("Outcome", axis=1)
y = df["Outcome"]

# -------------------------------
# Optional: Scale features
# -------------------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Save the scaler for future predictions
joblib.dump(scaler, "diabetes_scaler.pkl")

# -------------------------------
# Split data
# -------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# -------------------------------
# Train AutoML model
# -------------------------------
automl = AutoML()
automl_settings = {
    "time_budget": 60,          # seconds, adjust as needed
    "metric": 'accuracy',
    "task": 'classification',
    "log_file_name": "flaml_diabetes.log",
    "n_jobs": -1,                # use all available cores
}

automl.fit(X_train=X_train, y_train=y_train, **automl_settings)

# -------------------------------
# Evaluate model
# -------------------------------
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = automl.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"\nTest Accuracy: {acc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# -------------------------------
# Save trained model
# -------------------------------
joblib.dump(automl, "diabetes_automl_model.pkl")
print("\nAutoML model saved as 'diabetes_automl_model.pkl' and scaler as 'diabetes_scaler.pkl'.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 7.1 MB/s eta 0:00:00
Sample records:
   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  
[flaml.automl.logger: 10-15 12:49:35] {1752} INFO - task = classification
[flaml.automl.logger: 10-15 12:49:35] {1763} INFO - Evaluation method: cv
[flaml.automl.logger: 10-15 12:49:35] {1862} INFO - Minimiz

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/ut

[flaml.automl.logger: 10-15 12:49:35] {2466} INFO -  at 0.3s,	estimator sgd's best error=0.2150,	best estimator sgd's best error=0.2150
[flaml.automl.logger: 10-15 12:49:35] {2282} INFO - iteration 4, current learner lgbm
[flaml.automl.logger: 10-15 12:49:35] {2466} INFO -  at 0.4s,	estimator lgbm's best error=0.2345,	best estimator sgd's best error=0.2150
[flaml.automl.logger: 10-15 12:49:35] {2282} INFO - iteration 5, current learner lgbm
[flaml.automl.logger: 10-15 12:49:35] {2466} INFO -  at 0.5s,	estimator lgbm's best error=0.2345,	best estimator sgd's best error=0.2150
[flaml.automl.logger: 10-15 12:49:35] {2282} INFO - iteration 6, current learner lgbm
[flaml.automl.logger: 10-15 12:49:35] {2466} INFO -  at 0.5s,	estimator lgbm's best error=0.2345,	best estimator sgd's best error=0.2150
[flaml.automl.logger: 10-15 12:49:35] {2282} INFO - iteration 7, current learner lgbm


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/ut

[flaml.automl.logger: 10-15 12:49:35] {2466} INFO -  at 0.6s,	estimator lgbm's best error=0.2345,	best estimator sgd's best error=0.2150
[flaml.automl.logger: 10-15 12:49:35] {2282} INFO - iteration 8, current learner lgbm
[flaml.automl.logger: 10-15 12:49:35] {2466} INFO -  at 0.6s,	estimator lgbm's best error=0.2264,	best estimator sgd's best error=0.2150
[flaml.automl.logger: 10-15 12:49:35] {2282} INFO - iteration 9, current learner sgd
[flaml.automl.logger: 10-15 12:49:35] {2466} INFO -  at 0.7s,	estimator sgd's best error=0.2085,	best estimator sgd's best error=0.2085
[flaml.automl.logger: 10-15 12:49:35] {2282} INFO - iteration 10, current learner sgd


...

[flaml.automl.logger: 10-15 12:50:15] {2466} INFO -  at 40.9s,	estimator xgboost's best error=0.2199,	best estimator sgd's best error=0.2036
[flaml.automl.logger: 10-15 12:50:15] {2282} INFO - iteration 413, current learner xgboost
[flaml.automl.logger: 10-15 12:50:16] {2466} INFO -  at 40.9s,	estimator xgboost's best error=0.2199,	best estimator sgd's best error=0.2036
[flaml.automl.logger: 10-15 12:50:16] {2282} INFO - iteration 414, current learner lgbm
[flaml.automl.logger: 10-15 12:50:16] {2466} INFO -  at 41.0s,	estimator lgbm's best error=0.2231,	best estimator sgd's best error=0.2036
[flaml.automl.logger: 10-15 12:50:16] {2282} INFO - iteration 415, current learner sgd
[flaml.automl.logger: 10-15 12:50:16] {2466} INFO -  at 41.1s,	estimator sgd's best error=0.2036,	best estimator sgd's best error=0.2036
[flaml.automl.logger: 10-15 12:50:16] {2282} INFO - iteration 416, current learner xgboost
[flaml.automl.logger: 10-15 12:50:16] {2466} INFO -  at 41.2s,	estimator xgboost's bes

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[flaml.automl.logger: 10-15 12:50:16] {2466} INFO -  at 41.2s,	estimator sgd's best error=0.2036,	best estimator sgd's best error=0.2036
[flaml.automl.logger: 10-15 12:50:16] {2282} INFO - iteration 418, current learner sgd
[flaml.automl.logger: 10-15 12:50:16] {2466} INFO -  at 41.3s,	estimator sgd's best error=0.2036,	best estimator sgd's best error=0.2036
[flaml.automl.logger: 10-15 12:50:16] {2282} INFO - iteration 419, current learner sgd
[flaml.automl.logger: 10-15 12:50:16] {2466} INFO -  at 41.3s,	estimator sgd's best error=0.2036,	best estimator sgd's best error=0.2036
[flaml.automl.logger: 10-15 12:50:16] {2282} INFO - iteration 420, current learner sgd
[flaml.automl.logger: 10-15 12:50:16] {2466} INFO -  at 41.4s,	estimator sgd's best error=0.2036,	best estimator sgd's best error=0.2036
[flaml.automl.logger: 10-15 12:50:16] {2282} INFO - iteration 421, current learner xgb_limitdepth
[flaml.automl.logger: 10-15 12:50:16] {2466} INFO -  at 41.5s,	estimator xgb_limitdepth's bes

INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune


[flaml.automl.logger: 10-15 12:50:35] {2466} INFO -  at 60.0s,	estimator lrl1's best error=0.2150,	best estimator sgd's best error=0.2036
[flaml.automl.logger: 10-15 12:50:35] {2724} INFO - retrain sgd for 0.0s
[flaml.automl.logger: 10-15 12:50:35] {2727} INFO - retrained model: SGDClassifier(alpha=0.0023446642089786154, loss='log_loss', n_jobs=-1,
              tol=0.0001)
[flaml.automl.logger: 10-15 12:50:35] {2009} INFO - fit succeeded
[flaml.automl.logger: 10-15 12:50:35] {2010} INFO - Time taken to find the best model: 12.975502014160156

Test Accuracy: 0.7078

Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.82      0.78       100
           1       0.60      0.50      0.55        54

    accuracy                           0.71       154
   macro avg       0.68      0.66      0.67       154
weighted avg       0.70      0.71      0.70       154


Confusion Matrix:
[[82 18]
 [27 27]]

AutoML model saved as 'diabetes_automl

In [6]:
!pip install gradio pandas joblib scikit-learn

import gradio as gr
import pandas as pd
import numpy as np
import joblib

# -------------------------------
# Load model and scaler
# -------------------------------
automl = joblib.load("diabetes_automl_model.pkl")
scaler = joblib.load("diabetes_scaler.pkl")

# -------------------------------
# Prediction function
# -------------------------------
def predict_diabetes(Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin,
                     BMI, DiabetesPedigreeFunction, Age):

    # Prepare dataframe
    input_df = pd.DataFrame([[Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin,
                              BMI, DiabetesPedigreeFunction, Age]],
                            columns=['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin',
                                     'BMI','DiabetesPedigreeFunction','Age'])

    # Handle zeros in invalid columns
    cols_with_zero_invalid = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
    input_df[cols_with_zero_invalid] = input_df[cols_with_zero_invalid].replace(0, np.nan)
    input_df.fillna(input_df.median(), inplace=True)

    # Scale
    input_scaled = scaler.transform(input_df)

    # Predict
    pred = automl.predict(input_scaled)[0]
    prob = automl.predict_proba(input_scaled)[0][1]

    return f"Prediction: {'Diabetic' if pred==1 else 'Non-Diabetic'} (Probability: {prob:.2f})"

# -------------------------------
# Default input values
# -------------------------------
default_values = {
    "Pregnancies": 2,
    "Glucose": 120,
    "BloodPressure": 70,
    "SkinThickness": 25,
    "Insulin": 79,
    "BMI": 28.0,
    "DiabetesPedigreeFunction": 0.5,
    "Age": 30
}

# -------------------------------
# Custom CSS for background
# -------------------------------
css = f"""
body {{
    margin: 0;
    padding: 0;
    font-family: 'Segoe UI', sans-serif;
    background-image: url('/content/bg.jpeg');
    background-size: cover;
    background-repeat: no-repeat;
    background-attachment: fixed;
}}
h1 {{
    color: #ffffff;
    text-align: center;
    margin-bottom: 20px;
    text-shadow: 2px 2px 8px #000;
}}
.gradio-container {{
    max-width: 900px;
    margin: auto;
    padding: 25px;
    border-radius: 15px;
    background-color: rgba(255,255,255,0.85);
    box-shadow: 0 6px 18px rgba(0,0,0,0.35);
}}
label {{
    font-weight: 600;
    color: #333333;
}}
.gr-button {{
    background-color: #0d6efd;
    color: white;
    font-weight: 600;
    border-radius: 8px;
}}
.scroll-panel {{
    max-height: 400px;
    overflow-y: auto;
    padding-right: 10px;
}}
"""

# -------------------------------
# Build Gradio interface
# -------------------------------
with gr.Blocks(css=css) as demo:
    gr.HTML("<h1>🩺 Diabetes Prediction Using AutoML</h1>")

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### Input Patient Details")
            with gr.Accordion("Patient Details"): # Makes inputs collapsible
                pregnancies = gr.Number(label="Pregnancies", value=default_values["Pregnancies"])
                glucose = gr.Number(label="Glucose", value=default_values["Glucose"])
                bp = gr.Number(label="Blood Pressure", value=default_values["BloodPressure"])
                skin = gr.Number(label="Skin Thickness", value=default_values["SkinThickness"])
                insulin = gr.Number(label="Insulin", value=default_values["Insulin"])
                bmi = gr.Number(label="BMI", value=default_values["BMI"])
                dpf = gr.Number(label="Diabetes Pedigree Function", value=default_values["DiabetesPedigreeFunction"])
                age = gr.Number(label="Age", value=default_values["Age"])
            submit_btn = gr.Button("Submit", variant="primary")

        with gr.Column(scale=1):
            gr.Markdown("### Prediction Output")
            output_box = gr.Textbox(label="Result", lines=3, interactive=False)

    # Connect button click
    submit_btn.click(
        fn=predict_diabetes,
        inputs=[pregnancies, glucose, bp, skin, insulin, bmi, dpf, age],
        outputs=output_box
    )

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://08becdf07f17dff2a0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
